# Neural Networks image recognition - ConvNet

1. Add random noise (see below on `size parameter` on [`np.random.normal`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html)) to the images in training and testing. **Make sure each image gets a different noise feature added to it. Inspect by printing out several images. Note - the `size` parameter should match the data. **
2. Compare the `accuracy` of train and val after N epochs for MLNN with and without noise. 
3. Vary the amount of noise by changing the `scale` parameter in `np.random.normal` by a factor. Use `.1, .5, 1.0, 2.0, 4.0` for the `scale` and keep track of the `accuracy` for training and validation and plot these results.
4. Compare these results with the previous week where we used a MultiLayer Perceptron (this week we use a ConvNet). 

# Neural Networks - Image Recognition 

In [1]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend
import tensorflow as tf 

## Conv Net
Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).


In [21]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [22]:
x_train.shape, y_train.shape

((60000, 28, 28), (60000,))

In [23]:
if backend.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [24]:
x_train.shape

(60000, 28, 28, 1)

In [25]:
batch_size = 128
num_classes = 10
epochs = 12

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

In [26]:
y_train, y_test

(array([[0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.]], dtype=float32),
 array([[0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

In [27]:
x_train.shape, y_train.shape

((60000, 28, 28, 1), (60000, 10))

In [28]:
# define the model and evaluate without noise
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
469/469 [==============================] - 47s 100ms/step - loss: 2.2842 - accuracy: 0.1425 - val_loss: 2.2582 - val_accuracy: 0.2808
Epoch 2/12
469/469 [==============================] - 53s 113ms/step - loss: 2.2398 - accuracy: 0.2245 - val_loss: 2.2041 - val_accuracy: 0.3856
Epoch 3/12
469/469 [==============================] - 53s 112ms/step - loss: 2.1834 - accuracy: 0.3031 - val_loss: 2.1342 - val_accuracy: 0.4376
Epoch 4/12
469/469 [==============================] - 53s 112ms/step - loss: 2.1097 - accuracy: 0.3622 - val_loss: 2.0408 - val_accuracy: 0.5035
Epoch 5/12
469/469 [==============================] - 53s 113ms/step - loss: 2.0123 - accuracy: 0.4154 - val_loss: 1.9171 - val_accuracy: 0.5722
Epoch 6/12
469/469 [==============================] - 51s 110ms/step - loss: 1.8868 - accuracy: 0.4716 - val_loss: 1.7592 - val_accuracy: 0.6410
Epoch 7/12
469/469 [==============================] - 52s 111ms/step - loss: 1.7368 - accuracy: 0.5220 - val_loss: 1.5737 - val_ac

In [29]:
# create noisy data at scale 2
from numpy import random

rando = random.normal(loc=1, scale=2, size=x_train.shape)
x_train_n = x_train + rando

rando = random.normal(loc=1, scale=2, size=x_test.shape)
x_test_n = x_test + rando

In [ ]:
# define the model and evaluate with noise
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train_n, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test_n, y_test))
score = model.evaluate(x_test_n, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/12
469/469 [==============================] - 50s 106ms/step - loss: 2.4563 - accuracy: 0.1003 - val_loss: 2.3045 - val_accuracy: 0.1056
Epoch 2/12
469/469 [==============================] - 53s 114ms/step - loss: 2.3469 - accuracy: 0.1017 - val_loss: 2.2991 - val_accuracy: 0.1187
Epoch 3/12
469/469 [==============================] - 54s 115ms/step - loss: 2.3187 - accuracy: 0.1052 - val_loss: 2.3002 - val_accuracy: 0.1186
Epoch 4/12
469/469 [==============================] - 54s 115ms/step - loss: 2.3093 - accuracy: 0.1058 - val_loss: 2.3005 - val_accuracy: 0.1177
Epoch 5/12
 84/469 [====>.........................] - ETA: 44s - loss: 2.3051 - accuracy: 0.1098

In [12]:
# define the model and evaluate with noise at difference scales
scales = [.1, .5, 1.0, 2.0, 4.0]
scores_n = []


model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

for scale in scales:
        print(scale)
        
        x = random.normal(loc=1, scale=scale, size=x_train.shape)
        x_train = x_train + x
        
        x = random.normal(loc=1, scale=scale, size=x_test.shape)
        x_test = x_test + x

        model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
        score = model.evaluate(x_test, y_test, verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        
        scores_n.append(score)

0.1
Epoch 1/12
469/469 [==============================] - 45s 96ms/step - loss: 2.3094 - accuracy: 0.1130 - val_loss: 2.2666 - val_accuracy: 0.2899
Epoch 2/12
469/469 [==============================] - 50s 107ms/step - loss: 2.2730 - accuracy: 0.1460 - val_loss: 2.2317 - val_accuracy: 0.4553
Epoch 3/12
469/469 [==============================] - 48s 102ms/step - loss: 2.2476 - accuracy: 0.1760 - val_loss: 2.2015 - val_accuracy: 0.5333
Epoch 4/12
469/469 [==============================] - 49s 105ms/step - loss: 2.2200 - accuracy: 0.2055 - val_loss: 2.1688 - val_accuracy: 0.6012
Epoch 5/12
469/469 [==============================] - 50s 106ms/step - loss: 2.1916 - accuracy: 0.2358 - val_loss: 2.1331 - val_accuracy: 0.6613
Epoch 6/12
469/469 [==============================] - 49s 104ms/step - loss: 2.1597 - accuracy: 0.2614 - val_loss: 2.0916 - val_accuracy: 0.6904
Epoch 7/12
469/469 [==============================] - 50s 107ms/step - loss: 2.1223 - accuracy: 0.2944 - val_loss: 2.0438 - val

469/469 [==============================] - 87s 185ms/step - loss: 2.2535 - accuracy: 0.1669 - val_loss: 2.2408 - val_accuracy: 0.1875
Epoch 8/12
469/469 [==============================] - 79s 169ms/step - loss: 2.2531 - accuracy: 0.1635 - val_loss: 2.2402 - val_accuracy: 0.1884
Epoch 9/12
469/469 [==============================] - 72s 154ms/step - loss: 2.2541 - accuracy: 0.1664 - val_loss: 2.2397 - val_accuracy: 0.1889
Epoch 10/12
469/469 [==============================] - 63s 134ms/step - loss: 2.2502 - accuracy: 0.1662 - val_loss: 2.2374 - val_accuracy: 0.1910
Epoch 11/12
469/469 [==============================] - 46s 98ms/step - loss: 2.2504 - accuracy: 0.1675 - val_loss: 2.2380 - val_accuracy: 0.1934
Epoch 12/12
469/469 [==============================] - 46s 98ms/step - loss: 2.2486 - accuracy: 0.1683 - val_loss: 2.2362 - val_accuracy: 0.1898
Test loss: 2.2362260818481445
Test accuracy: 0.18979999423027039


In [13]:
scores_n

[[1.73496675491333, 0.7680000066757202],
 [1.417175054550171, 0.743399977684021],
 [1.4804304838180542, 0.6202999949455261],
 [1.9089007377624512, 0.3799000084400177],
 [2.2362260818481445, 0.18979999423027039]]

In [ ]:
plt.figure()
plt.plot(scales, train_accuracy, label = 'Train', c = "Red")
plt.plot(scales, test_accuracy, label = 'Test', c = "Blue")
plt.xlabel('Scale')
plt.ylabel('Accuracy')
plt.title('Accuracy of Image Model With Noise')
plt.legend()
plt.show()